In [1]:
# Required python package imports
import sqlalchemy
import sys
from sqlalchemy import create_engine, MetaData, Table, select
import pyodbc
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
import numpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import time 
import multiprocessing 
%matplotlib inline
import matplotlib.image as image
import datetime

In [76]:
port_raw=pd.read_csv('C:/Users/ywu/Google Drive/Future/Sample/Portfolio Analytics/Sample_Holdings_20160624.csv', header=0)
port_raw.columns=['Ticker','Sedol','Name','PortID','SecType','Pos','Cur','Country','Shares','px_prev','px','dv','alpha','mv','weight']

In [79]:
port_raw['wgt']=port_raw.mv/port_raw.mv.sum()
port_raw['wgt_ctry_ls']=port_raw.groupby(['Country','Pos'])['mv'].apply(lambda x : x/x.sum())
port_raw['ret']= port_raw['px']/port_raw['px_prev']-1
port_raw['ret_contr'] = port_raw['wgt'] *  port_raw['ret'] 

In [81]:
port_raw.ret_contr.sum()

-0.05457462295131797

In [80]:
port_raw

,Ticker,Sedol,Name,PortID,SecType,Pos,Cur,Country,Shares,px_prev,px,dv,alpha,mv,weight,wgt,wgt_ctry_ls,ret,ret_contr
0,ADS,4031976,ADIDAS AG,45,EQTY,S,EUR,EU,-30.0,123.50,119.85,0.85,-0.05,-3595.50,-2.72%,-0.000167,0.027183,-0.029555,4.924233e-06
1,ADV,5653487,ADVA OPTICAL NETWG.AG,45,EQTY,L,EUR,EU,710.0,8.49,7.79,0.95,-0.03,5531.61,2.56%,0.000256,0.025635,-0.082450,-2.113466e-05
2,ALV,5231485,ALLIANZ SE,45,EQTY,S,EUR,EU,-30.0,141.50,126.80,0.61,-0.10,-3804.00,-2.88%,-0.000176,0.028759,-0.103887,1.831280e-05
3,AOX,B1VP947,ALSTRIA OFFICE REIT AG,45,EQTY,L,EUR,EU,620.0,12.08,11.94,0.84,0.05,7402.80,3.43%,0.000343,0.034306,-0.011589,-3.975671e-06
4,AR4,B17NLM7,AURELIUS SE & CO.KGAA,45,EQTY,L,EUR,EU,130.0,53.60,51.54,0.58,0.02,6700.20,3.11%,0.000310,0.031050,-0.038433,-1.193283e-05
5,ARL,7380062,AAREAL BANK AG,45,EQTY,L,EUR,EU,230.0,31.76,29.00,0.81,0.04,6670.00,3.09%,0.000309,0.030910,-0.086902,-2.686011e-05
6,BAS,5086577,BASF SE,45,EQTY,S,EUR,EU,-50.0,71.87,67.07,0.66,-0.04,-3353.50,-2.54%,-0.000155,0.025353,-0.066787,1.037876e-05
7,BC8,5932409,BECHTLE AG,45,EQTY,S,EUR,EU,-10.0,97.40,94.00,0.46,0.08,-940.00,-0.71%,-0.000044,0.007107,-0.034908,1.520552e-06
8,BDT,5130861,BERTRANDT AG,45,EQTY,L,EUR,EU,60.0,94.79,89.80,0.59,-0.01,5388.00,2.50%,0.000250,0.024969,-0.052643,-1.314374e-05
9,BEI,5107401,BEIERSDORF AG,45,EQTY,S,EUR,EU,-40.0,81.31,80.11,0.55,-0.03,-3204.40,-2.42%,-0.000148,0.024226,-0.014758,2.191479e-06


In [75]:
port_raw['px']

0       119.85
1         7.79
2        126.8
3        11.94
4        51.54
5           29
6        67.07
7           94
8         89.8
9        80.11
10       68.66
11        3.92
12       27.05
13         6.2
14      181.25
15       35.63
16       62.52
17       55.06
18       13.37
19       24.56
20       35.34
21       55.41
22       14.06
23        3.67
24       63.14
25        22.9
26        27.1
27       19.15
28       42.18
29       40.79
         ...  
1046     52.12
1047     51.29
1048     73.62
1049      89.6
1050     49.68
1051     72.24
1052     26.15
1053     34.37
1054     76.66
1055    100.72
1056    110.34
1057     61.61
1058     98.08
1059     69.35
1060     74.66
1061      69.6
1062    111.02
1063     75.28
1064     70.72
1065      41.6
1066     37.38
1067     85.25
1068     61.31
1069     52.56
1070     96.99
1071     47.84
1072    162.57
1073     71.96
1074     18.61
1075     36.24
Name: px, dtype: object

In [66]:
port_raw 

,Ticker,Sedol,Name,PortID,SecType,Pos,Cur,Country,Shares,px_prev,px,dv,alpha,mv,weight,wgt
0,ADS,4031976,ADIDAS AG,45,EQTY,S,EUR,EU,-30,123.5,119.85,0.848130,-0.046947,-3595.50,-2.72%,-0.000167
1,ADV,5653487,ADVA OPTICAL NETWG.AG,45,EQTY,L,EUR,EU,710,8.49,7.79,0.951086,-0.027181,5531.61,2.56%,0.000256
2,ALV,5231485,ALLIANZ SE,45,EQTY,S,EUR,EU,-30,141.5,126.8,0.605286,-0.102125,-3804.00,-2.88%,-0.000176
3,AOX,B1VP947,ALSTRIA OFFICE REIT AG,45,EQTY,L,EUR,EU,620,12.08,11.94,0.836596,0.048959,7402.80,3.43%,0.000343
4,AR4,B17NLM7,AURELIUS SE & CO.KGAA,45,EQTY,L,EUR,EU,130,53.6,51.54,0.582173,0.019275,6700.20,3.11%,0.000310
5,ARL,7380062,AAREAL BANK AG,45,EQTY,L,EUR,EU,230,31.76,29,0.805694,0.037407,6670.00,3.09%,0.000309
6,BAS,5086577,BASF SE,45,EQTY,S,EUR,EU,-50,71.87,67.07,0.663451,-0.038667,-3353.50,-2.54%,-0.000155
7,BC8,5932409,BECHTLE AG,45,EQTY,S,EUR,EU,-10,97.4,94,0.460524,0.076107,-940.00,-0.71%,-0.000044
8,BDT,5130861,BERTRANDT AG,45,EQTY,L,EUR,EU,60,94.79,89.8,0.594834,-0.014579,5388.00,2.50%,0.000250
9,BEI,5107401,BEIERSDORF AG,45,EQTY,S,EUR,EU,-40,81.31,80.11,0.547442,-0.026088,-3204.40,-2.42%,-0.000148


In [60]:
port_raw.mv.sum()

'-3595.55531.61-38047402.86700.26670-3353.5-9405388-3204.4-2059.87244.6-3245.4-2542.41-36257838.66877.21101.2935.55491.1-1413.6-554.1-3232.65-330.3-631.4-458-2980.45-574.35-421.82447.4717.3-1047-2164.81110.5844.54163.25-277.7-547.751244.5915.3422.25-511.4506.11679-443.05-854.1-393.051961.540281121-499.86633527.32115-315.75-3620.8469.452748-576.5776.85377.9-3295053.25-354-380.55520.4806.41532-503.55-554.85-2048-2899.6-803.3554543.56834-3048.75-3303.45-3697.58239-3322.9-3272.4-584504.6-1150-772.9-3429.4-6112867.051224.440882400-722.9-522.1-1421.5-3095.1-825.45-381.93658-2850.91351.5-24666351.8-702.76821.25-606.6-736.5-770.8-951.73252-704.9-651.6-693.7-3202.21952.6-307.5-1291.22411.6325.9585.5-779.75404.3415.77221.9-2989.9-333564786060.357555.1-20006232.17459.5-3141.6467.46910.2-534.8-1634-517-1498-2700-397.2-636.2-2402.4-697.5729.126211.8-453-332.4690.3-467.2611.6-509-1198.8-390.5-569.75-2402-497.61139.882885.64-298512744-5909-6214919.51378448814157130901269214360111069113.67869900-66001

In [58]:
port_raw.mv

0           -3595.5
1           5531.61
2             -3804
3            7402.8
4            6700.2
5              6670
6           -3353.5
7              -940
8              5388
9           -3204.4
10          -2059.8
11           7244.6
12          -3245.4
13         -2542.41
14            -3625
15           7838.6
16           6877.2
17           1101.2
18           935.55
19            491.1
20          -1413.6
21           -554.1
22         -3232.65
23           -330.3
24           -631.4
25             -458
26         -2980.45
27          -574.35
28           -421.8
29           2447.4
           ...     
1046      4,169.60 
1047     (7,180.60)
1048     (5,889.60)
1049      5,376.00 
1050     (7,452.00)
1051     (7,224.00)
1052     (6,276.00)
1053     (3,093.30)
1054      6,899.40 
1055     (5,036.00)
1056      3,310.20 
1057      2,464.40 
1058     (1,961.60)
1059      3,467.50 
1060      2,239.80 
1061      6,264.00 
1062      6,661.20 
1063           3764
1064        -4950.4


In [ ]:
df.port_raw..apply(lambda x: x / x.sum())

df.iloc[:, 3:].apply(lambda x: x / x.sum())

In [50]:
pd_total=pd.DataFrame.from_items([
    ('name', ['JIM','JIM','BOB','BOB','TIM','TIM']), 
    ('PET', ['CAT','DOG','CAT','DOG','CAT','DOG']),
    ('total', [12,253,12,234,52,23])
])

In [51]:
pd_total

,name,PET,total
0,JIM,CAT,12
1,JIM,DOG,253
2,BOB,CAT,12
3,BOB,DOG,234
4,TIM,CAT,52
5,TIM,DOG,23


In [52]:
pd_total.total.pct_change

<bound method NDFrame.pct_change of 0     12
1    253
2     12
3    234
4     52
5     23
Name: total, dtype: int64>

In [48]:
 port_raw.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))

TypeError: cannot convert the series to <class 'float'>

In [33]:
port_raw.mv.sum()

'-3595.55531.61-38047402.86700.26670-3353.5-9405388-3204.4-2059.87244.6-3245.4-2542.41-36257838.66877.21101.2935.55491.1-1413.6-554.1-3232.65-330.3-631.4-458-2980.45-574.35-421.82447.4717.3-1047-2164.81110.5844.54163.25-277.7-547.751244.5915.3422.25-511.4506.11679-443.05-854.1-393.051961.540281121-499.86633527.32115-315.75-3620.8469.452748-576.5776.85377.9-3295053.25-354-380.55520.4806.41532-503.55-554.85-2048-2899.6-803.3554543.56834-3048.75-3303.45-3697.58239-3322.9-3272.4-584504.6-1150-772.9-3429.4-6112867.051224.440882400-722.9-522.1-1421.5-3095.1-825.45-381.93658-2850.91351.5-24666351.8-702.76821.25-606.6-736.5-770.8-951.73252-704.9-651.6-693.7-3202.21952.6-307.5-1291.22411.6325.9585.5-779.75404.3415.77221.9-2989.9-333564786060.357555.1-20006232.17459.5-3141.6467.46910.2-534.8-1634-517-1498-2700-397.2-636.2-2402.4-697.5729.126211.8-453-332.4690.3-467.2611.6-509-1198.8-390.5-569.75-2402-497.61139.882885.64-298512744-5909-6214919.51378448814157130901269214360111069113.67869900-66001

In [42]:
dt = np.array (['20150101','20150101','20150102','20150102'])
nm = np.array(['A','B','A','B'])
val = np.array([10,5,2,10])

In [ ]:
df

In [40]:
dt


array(['20150101', '20150101', '20150102', '20150102'], 
      dtype='<U8')

In [41]:
nm 

array(['A', 'A', 'B', 'B'], 
      dtype='<U1')

In [38]:

asq = np.array(np.array([10,10,20]))
columns=['asp']
df = pd.DataFrame(asp, index=None, columns=columns)
df['asq'] = asq
df = df.groupby(by=['asp']).sum()
dfcum =df.cumsum()

array([0, 0, 1])

In [35]:
dfcum

,asq
asp,
0,20
1,40


In [36]:
df

,asq
asp,
0,20
1,20


In [47]:
df = pd.DataFrame({'state': ['CA', 'WA', 'CO', 'AZ'] * 3,
               'office_id': range(1, 7) * 2,
               'sales': [np.random.randint(100000, 999999) ]})

df.groupby(['state', 'office_id']).agg({'sales': 'sum'})

TypeError: unsupported operand type(s) for *: 'range' and 'int'

In [44]:
state_pcts = state_office.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))

NameError: name 'state_office' is not defined